In [1]:
# Your libraries here
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [31]:
# change encoding bcs utf-8 does not work
df = pd.read_csv("HOBO/Group3-Mid.csv", encoding="iso-8859-9")
display(df)

df2= pd.read_excel("CALERA/G3-OO-HAND.xlsx")
display(df2)

df3= pd.read_excel("CUBE/Group 3-Cube.xlsx")
display(df3)

df4= pd.read_excel("Relative Humidity.xlsx")
display(df4)


,Date & Time,T_air (°C),T_globe (°C),Air_speed (m/s)
0,3/14/2025 9:15,28.731,28.995,0.028590
1,3/14/2025 9:15,28.731,28.997,0.000239
2,3/14/2025 9:15,28.731,28.997,0.000162
3,3/14/2025 9:15,28.732,28.997,0.000162
4,3/14/2025 9:15,28.732,28.997,0.000162
...,...,...,...,...
7196,3/14/2025 11:14,19.884,20.405,0.091410
7197,3/14/2025 11:14,19.882,20.405,0.138889
7198,3/14/2025 11:14,19.879,20.403,0.135120
7199,3/14/2025 11:14,19.877,20.403,0.112538


,Date & Time,Heat Flux (W/m2),T_sk (°C),T_core (°C)
0,2025-03-14 09:15:00.000,46.796185,31.677,37.000
1,2025-03-14 09:15:01.000,46.796185,31.681,37.000
2,2025-03-14 09:15:02.000,46.796185,31.683,37.000
3,2025-03-14 09:15:03.010,46.796185,31.687,37.000
4,2025-03-14 09:15:04.015,46.796185,31.689,37.000
...,...,...,...,...
7160,2025-03-14 11:14:55.795,100.981242,29.093,37.305
7161,2025-03-14 11:14:56.800,100.981242,29.097,37.305
7162,2025-03-14 11:14:57.805,100.981242,29.097,37.305
7163,2025-03-14 11:14:58.810,100.981242,29.097,37.305


,DD-MM-YYYY,HH:MM:SS,Mean Radiant Temperature (*C),UP LW IRRAD (*C),NORTH LW IRRAD (*C),EAST LW IRRAD (*C),SOUTH LW IRRAD (*C),WEST LW IRRAD (*C),DOWN LW IRRAD (*C)
0,14-03-2025,09:15:00,29.58,29.15,29.85,30.16,30.04,29.31,28.97
1,14-03-2025,09:15:01,29.57,29.17,29.88,30.15,30.03,29.29,28.90
2,14-03-2025,09:15:02,29.54,29.15,29.85,30.11,29.93,29.29,28.89
3,14-03-2025,09:15:04,29.56,29.15,29.85,30.16,29.93,29.39,28.85
4,14-03-2025,09:15:05,29.56,29.23,29.83,30.14,29.93,29.31,28.91
...,...,...,...,...,...,...,...,...,...
5368,14-03-2025,11:14:55,22.17,20.82,22.35,21.91,24.51,22.77,20.65
5369,14-03-2025,11:14:56,22.20,20.77,22.37,21.95,24.51,22.95,20.67
5370,14-03-2025,11:14:58,22.18,20.77,22.25,21.85,24.57,22.89,20.73
5371,14-03-2025,11:14:59,22.18,20.77,22.28,21.85,24.63,22.81,20.76


,Date Time,"RH, %"
0,2025-03-14 10:25:19,46.581
1,2025-03-14 10:25:20,46.615
2,2025-03-14 10:25:21,46.647
3,2025-03-14 10:25:22,46.710
4,2025-03-14 10:25:23,46.744
...,...,...
3036,2025-03-14 11:15:55,33.983
3037,2025-03-14 11:15:56,33.983
3038,2025-03-14 11:15:57,33.983
3039,2025-03-14 11:15:58,33.983


In [16]:
# convective res
M = 58.
ta = df["T_air (°C)"]
q_conv_res = 0.0014 * M * (34-ta)
display(q_conv_res)

0       0.427843
1       0.427843
2       0.427843
3       0.427762
4       0.427762
          ...   
7196    1.146219
7197    1.146382
7198    1.146625
7199    1.146788
7200    1.147031
Name: T_air (°C), Length: 7201, dtype: float64

In [3]:
# functions
def calculate_f_cl(I_cl):
    """
    Calculates clothing factor (f_cl).
    I_cl: Clothing insulation [clo]
    """
    if I_cl <= 0.5:
        f_cl = 1 + 0.31 * I_cl
    else:
        f_cl = 1.05 + 0.645 * I_cl
    return f_cl


In [4]:
calculate_f_cl(0.38)

1.1178

In [10]:
import numpy as np

def iterate_T_cl(T_skin, T_air, T_mrt, I_cl, f_cl, epsilon=0.95, max_iter=100, tol=0.01):
    """
    Iterative çözüm ile clothing temperature (T_cl) ve h_rad hesaplanır.
    Tüm sıcaklıklar [°C], çıktı T_cl [°C], h_rad [W/m2K]
    """
    # Başlangıç tahmini
    T_cl = (T_skin + T_air) / 2
    sigma = 5.67e-8  # Stefan-Boltzmann constant
    R_cl = I_cl * 0.155  # [m2K/W]
    f_eff = 0.72

    for i in range(max_iter):
        T_cl_K = T_cl + 273.15
        T_mrt_K = T_mrt + 273.15

        # h_rad hesapla
        h_rad = 4 * epsilon * sigma * f_eff * ((T_cl_K + T_mrt_K) / 2)**3

        # Combined heat transfer coefficient
        h_conv = 3.1  # default for moderate air speed (can be improved)
        # h_total = h_c + h_rad

        # Yeni T_cl hesapla (Eq. 2-11 rearranged)
        # T_cl_new = T_skin - ( (T_skin - T_air) + (T_skin - T_mrt) ) / (1 + f_cl * h_total * R_cl)
        T_cl_new = T_skin - R_cl * (h_conv * f_cl * (T_cl - T_air) + h_rad * f_cl * (T_cl - T_mrt))

        # Hata kontrolü
        if abs(T_cl_new - T_cl) < tol:
            break
        T_cl = T_cl_new

    return T_cl, h_rad

In [11]:
iterate_T_cl(32, 25, 23.5, 0.38, 1.1178, epsilon=0.95, max_iter=100, tol=0.01)

(29.460962563315558, 4.173084008001793)

In [19]:
def Q_rad(h_rad, f_cl, T_cl, T_mrt):
    """
    Radiative heat loss from the body.
    T_cl, T_mrt: [°C]
    Returns Q_rad in [W/m²]
    """
    return h_rad * f_cl * (T_cl - T_mrt)

In [11]:
Q_rad(5.636, 1.1178, 23.917, 23.5)

2.62706697360001

In [20]:
def Q_conv_skin(h_c, f_cl, T_cl, T_air):
    """
    Convective heat loss from clothing surface.
    Returns Q_conv_skin in [W/m²]
    """
    return h_c * f_cl * (T_cl - T_air)

In [13]:
Q_conv_skin(3.1, 1.1178, 23.917, 25)

-3.752789939999994

In [26]:
import numpy as np

def iterate_T_cl(T_skin, T_air, T_mrt, I_cl, f_cl, epsilon=0.95, max_iter=100, tol=0.01):
    """
    Iterative çözüm ile clothing temperature (T_cl) ve h_rad hesaplanır.
    Tüm sıcaklıklar [°C], çıktı T_cl [°C], h_rad [W/m2K]
    """
    # Başlangıç tahmini
    T_cl = (T_skin + T_air) / 2
    sigma = 5.67e-8  # Stefan-Boltzmann constant
    R_cl = I_cl * 0.155  # [m2K/W]
    f_eff = 0.72

    for i in range(max_iter):
        T_cl_K = T_cl + 273.15
        T_mrt_K = T_mrt + 273.15

        # h_rad hesapla
        h_rad = 4 * epsilon * sigma * f_eff * ((T_cl_K + T_mrt_K) / 2)**3

        # Combined heat transfer coefficient
        h_conv = 3.1  # default for moderate air speed (can be improved)
        # h_total = h_c + h_rad

        # Yeni T_cl hesapla (Eq. 2-11 rearranged)
        # T_cl_new = T_skin - ( (T_skin - T_air) + (T_skin - T_mrt) ) / (1 + f_cl * h_total * R_cl)
        # T_cl_new = T_skin - R_cl * (h_conv * f_cl * (T_cl - T_air) + h_rad * f_cl * (T_cl - T_mrt))

        Q_radi = Q_rad(h_rad, f_cl, T_cl, T_mrt)
        Q_conv = Q_conv_skin(h_conv, f_cl, T_cl, T_air)

        T_cl_new = T_skin - R_cl * (Q_radi + Q_conv)

        # Hata kontrolü
        if abs(T_cl_new - T_cl) < tol:
            break
        T_cl = T_cl_new

    return T_cl, h_rad, Q_radi, Q_conv

In [27]:
iterate_T_cl(32, 25, 23.5, 0.38, 1.1178, epsilon=0.95, max_iter=100, tol=0.01)

(29.460962563315558, 4.173084008001793, 27.805942936102284, 15.458038255149804)

In [14]:
def Q_conv_resp(M, T_air):
    """
    Convective heat loss via respiration.
    M: metabolic rate [W/m²]
    Returns Q_conv_resp in [W/m²]
    """
    return 0.0014 * M * (34 - T_air)

In [19]:
# convective res
M = 58.2
ta = df["T_air (°C)"]
q_conv_res = Q_conv_resp(M, ta)
display(q_conv_res)

0       0.429318
1       0.429318
2       0.429318
3       0.429237
4       0.429237
          ...   
7196    1.150172
7197    1.150335
7198    1.150579
7199    1.150742
7200    1.150986
Name: T_air (°C), Length: 7201, dtype: float64

In [ ]:
def calculate_Q_evap_skin(w, p_sat_skin, p_air, R_cl, f_cl, h_conv, LR=16.5):
    """
    Evaporative heat loss from the skin surface
    All pressures in Pa
    Returns Q_E_skin in W/m²
    """
    h_e = LR * h_conv
    resistance = R_cl + 1 / (f_cl * h_e)
    return w * (p_sat_skin - p_air) / resistance

def calculate_Q_evap_resp(M, p_air_kPa):
    """
    Evaporative heat loss via respiration
    p_air in [kPa]
    Returns Q_E_resp in W/m²
    """
    return 0.001 * M * (2.8 - p_air_kPa)